In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import labrotation.file_handling as fh
import seaborn as sns
import os
from datetime import datetime
import datadoc_util
import h5py
import numpy as np
from math import floor, ceil
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from scipy import signal

In [ ]:
save_figs = False

In [ ]:
sns.set_theme(font_scale=2)
sns.set_style("whitegrid")

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
if "DATA_DOCU_FOLDER" in env_dict.keys():
    docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    docu_folder = fh.open_dir("Choose folder containing folders for each mouse!")
print(f"Selected folder:\n\t{docu_folder}")

In [ ]:
if "documentation" in os.listdir(docu_folder):
    mouse_folder = os.path.join(docu_folder, "documentation")
else:
    mouse_folder = docu_folder
mouse_names = os.listdir(mouse_folder)
print(f"Mice detected:")
for mouse in mouse_names:
    print(f"\t{mouse}")

In [ ]:
def get_datetime_for_fname():
    now = datetime.now()
    return f"{now.year:04d}{now.month:02d}{now.day:02d}-{now.hour:02d}{now.minute:02d}{now.second:02d}"

In [ ]:
output_folder = env_dict["DOWNLOADS_FOLDER"]
print(f"Output files will be saved to {output_folder}")

In [ ]:
ddoc = datadoc_util.DataDocumentation(docu_folder)
ddoc.loadDataDoc()

In [ ]:
chr2_fpath = fh.open_file("Open ChR2 assembled traces h5 file!")

In [ ]:
tmev_fpath = fh.open_file("Open TMEV assembled traces h5 file!")

# Open mean fluorescence, segments

In [ ]:
dict_uuid_mean_fluo = dict()
dict_uuid_meta = dict()
with h5py.File(tmev_fpath, "r") as hf:
    for uuid in hf.keys():
        dict_attrs = dict()
        for attr_name in hf[uuid].attrs.keys():
            dict_attrs[attr_name] = hf[uuid].attrs[attr_name]
        dict_uuid_meta[uuid] = dict_attrs
        mean_fluo = hf[uuid]["mean_fluo"][()]
        dict_uuid_mean_fluo[uuid] = mean_fluo
with h5py.File(chr2_fpath, "r") as hf:
    for uuid in hf.keys():
        dict_attrs = dict()
        for attr_name in hf[uuid].attrs.keys():
            dict_attrs[attr_name] = hf[uuid].attrs[attr_name]
        dict_uuid_meta[uuid] = dict_attrs
        mean_fluo = hf[uuid]["mean_fluo"][()]
        dict_uuid_mean_fluo[uuid] = mean_fluo

In [ ]:
# using scipy 1.7.3 right now... ShortTimeFFT.spectrogram would be more modern option

In [ ]:
dict_uuid_meta

In [ ]:
fs = 15.  # ~15 Hz imaging
example_uuid = "d43a110d015345c58ed961ee510e70aa"
example_trace = dict_uuid_mean_fluo[example_uuid]
if "i_stim_begin_frame" in dict_uuid_meta[example_uuid]:
    i_stim_begin = dict_uuid_meta[example_uuid]["i_stim_begin_frame"]
    i_stim_end = dict_uuid_meta[example_uuid]["i_stim_end_frame"]
    example_trace[i_stim_begin:i_stim_end+1] = np.mean(example_trace)
i_begin_frame = 0

In [ ]:
f, t, Sxx = signal.spectrogram(example_trace[i_begin_frame:], fs)
tsteps = np.array([i/fs for i in range(len(example_trace[i_begin_frame:]))])

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(18,18), sharex=True)
axs[0].pcolormesh(t, f, Sxx, shading='gouraud')
axs[0].set_ylim((0, 3))
plt.rcParams['axes.grid'] = False 
plt.xlabel('Time [sec]')
plt.xlim((250, 450))
#plt.xlim((0, 100))
axs[0].set_ylabel("Frequency [Hz]")
axs[1].plot(tsteps, example_trace[i_begin_frame:])
axs[1].set_ylabel("FOV mean fluorescence (a.u.)")

#plt.ylim((0, 3))
plt.tight_layout()
if save_figs:
    fig_fpath = os.path.join(output_folder, f'spectrogram_example_{get_datetime_for_fname()}.pdf')
    plt.savefig(fig_fpath)
    print(f"Saved to {fig_fpath}")  
else:
    plt.show()